# Pràctica 2: Recomanador Simple

Nom dels alumnes del grup:


## 1. INTRODUCCIÓ

### 1.1. Abans de començar...

**\+ A més a més de les que ja es troben presents en la 1a cel·la i funcions natives de Python, durant la pràctica, només es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica què serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades.

### 1.2. Dades: puntuacions de pel·licules

La base de dades [movielens-1M](http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·lícules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online [MovieLens](http://www.movielens.org/). 

El consum total de tots els usuaris s'hi pot trobar al document "ratings.dat" el format següent:

    UserID::MovieID::Rating::Timestamp

- **UserID** usuari, amb id's entre 1 i 6040 
- **MovieID** pel·licula, amb id's entre 1 i 3952
- **Rating** puntuació, en una escala de 1 a 5 estrelles.
- **Timestamp** representat en segons

Cada usuari té com a mínim 20 puntuacions.

### 1.3. Dades: usuaris



Al fitxer ``users.dat`` hi trobem la informació referent a cadascun dels usuaris en el següent format:

        UserID::Gender::Age::Occupation::Zip-code

- **Gender** ve donat per "M" per home i "F" per dona.
- **Age** està representada de la següent forma:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- **Occupation** es tria entre les següents opcions:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

Els usuaris han donat la informació voluntariament. Així doncs, alguns usuaris poden no tenir informació.


### 1.4. Dades: pel·lícules



Al fitxer ``movies.dat`` hi trobem la informació referent a cadascuna de les películes en el següent format:

        MovieID::Title::Genres

- **Titles** són identics als titols de la base de dades IMDB, incloent l'any de llançament.
- **Genres** de les películes estan separats i seleccionats d'entre els següents:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

Algunes películes poden tenir l'ID malament degut a duplicats accidentals.

Les películes s'han entrat manualment, així que poden existir altres inconsistencies. 

## 2. Exploració de les dades

### 2.1. Descarregar i llegir dades

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball. 

In [1]:
import os
if os.path.isfile("/etc/password.txt") == False:
    os.system('curl -O  http://files.grouplens.org/datasets/movielens/ml-1m.zip')
    os.system('unzip ml-1m.zip')

+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [2]:
import math
import numpy as np
import pandas as pd
import datetime
import itertools
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

C:\Users\diego\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='latin-1')


### 2.2. Inspecció de les taules

In [4]:
users[:10]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [5]:
users[-10:]

,user_id,gender,age,occupation,zip
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [6]:
ratings[-10:]

,user_id,movie_id,rating,timestamp
1000199,6040,2022,5,956716207
1000200,6040,2028,5,956704519
1000201,6040,1080,4,957717322
1000202,6040,1089,4,956704996
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [7]:
ratings[:10]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [8]:
ratings.sort_values('movie_id')[:5]


,user_id,movie_id,rating,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317


In [9]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 2.3 **Exemple:** Com extreure informació d'un DataFrame.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**, dades que estan a frames diferents.

El primer pas a obtenir una única estructura que contingui tota la informació. Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció.

Reviseu aquests conceptes de pandas: https://pandas.pydata.org/docs/user_guide/merging.html

In [11]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificar d'usuari
data.sort_values(by='user_id')[:10]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
28501,1,48,5,978824351,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance
13819,1,938,4,978301752,F,1,10,48067,Gigi (1958),Musical
51327,1,1207,4,978300719,F,1,10,48067,To Kill a Mockingbird (1962),Drama
31152,1,1721,4,978300055,F,1,10,48067,Titanic (1997),Drama|Romance
37916,1,2762,4,978302091,F,1,10,48067,"Sixth Sense, The (1999)",Thriller
18472,1,2687,3,978824268,F,1,10,48067,Tarzan (1999),Animation|Children's
45685,1,2692,4,978301570,F,1,10,48067,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance
22832,1,720,3,978300760,F,1,10,48067,Wallace & Gromit: The Best of Aardman Animatio...,Animation
32771,1,745,3,978824268,F,1,10,48067,"Close Shave, A (1995)",Animation|Comedy|Thriller


La funció ``iloc`` ens permet obtenir un subconjunt de files i/o columnes indexades per un enter:

In [12]:
data.iloc[3:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


Els índexs Booleans ens permeten seleccionar una part de la taula que compleix una condició.

In [13]:
# comptem quin tant per cent de ratings estan fets per una dona

print(data[data['gender']=='F']['rating'].count()/float(data['rating'].count())*100, '%')

24.638850480249626 %


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes.

Reviseu aquests conceptes: 
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table

In [14]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

age,1,18,25,35,45,50,56
title,,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000,NaN
'Night Mother (1986),2.000000,4.666667,3.423077,2.904762,3.833333,3.555556,4.333333
'Til There Was You (1997),3.500000,2.500000,2.666667,2.900000,2.333333,2.500000,2.666667
"'burbs, The (1989)",4.500000,3.244444,2.652174,2.818182,2.545455,3.208333,2.666667
...And Justice for All (1979),3.000000,3.428571,3.724138,3.657143,4.100000,3.551724,3.928571
1-900 (1994),NaN,NaN,2.000000,NaN,NaN,NaN,3.000000
10 Things I Hate About You (1999),3.745455,3.415020,3.432950,3.102941,3.258065,3.629630,4.000000
101 Dalmatians (1961),3.514286,3.295082,3.613757,3.826087,3.976744,3.650000,3.190476
101 Dalmatians (1996),3.088235,2.467742,2.928571,3.279570,3.482759,3.400000,3.555556


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [15]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
1-900 (1994),2.000000,3.000000
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215


Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()`` el nombre.

Reviseu aquest concepte: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [16]:
ratings_by_title = data.groupby('title').size()
print(ratings_by_title)

title
$1,000,000 Duck (1971)                         37
'Night Mother (1986)                           70
'Til There Was You (1997)                      52
'burbs, The (1989)                            303
...And Justice for All (1979)                 199
                                             ... 
Zed & Two Noughts, A (1985)                    29
Zero Effect (1998)                            301
Zero Kelvin (Kjærlighetens kjøtere) (1995)      2
Zeus and Roxanne (1997)                        23
eXistenZ (1999)                               410
Length: 3706, dtype: int64


Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [17]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``: 

In [18]:
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [19]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [20]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [21]:
print(np.nan + 9.0) 

nan


Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre homes i dones:

In [22]:
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:15]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones: 

In [23]:
sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions: 

In [24]:
# Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std()
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

### Important: Temes de rendiment

Fixeu-vos en el comportament de Python i actueu en conseqüència:

In [25]:
%timeit data['title'] 
print(type(data['title']))
%timeit data.title 
print(type(data.title))
%timeit data[['title']] 
print(type(data[['title']]))

8.46 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
<class 'pandas.core.series.Series'>
20 µs ± 480 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
<class 'pandas.core.series.Series'>
47 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
<class 'pandas.core.frame.DataFrame'>


## 3. EXERCICIS

### 3.1. EXERCICI A

+ Donada la taula ``data`` tal i com es defineix a continuació, calcula la puntuació mitjana de cada usuari i guarda-la a un ``df`` anomenat ``users_mean_rating``. 

In [25]:
data_folder = 'ml-1m'

In [26]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(f'{data_folder}/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(f'{data_folder}/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(f'{data_folder}/movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='latin-1')

data = pd.merge(pd.merge(ratings, users), movies)

# la vostra solució aquí
users_mean_rating = data.pivot_table(values= 'rating', index='user_id', aggfunc='mean')


users_mean_rating

,rating
user_id,
1,4.188679
2,3.713178
3,3.901961
4,4.190476
5,3.146465
...,...
6036,3.302928
6037,3.717822
6038,3.800000


+ Quina és la pel·lícula més ben puntuada (en mitja) pels usuaris? (Guarda aquest valor en una variable de tipus ``string`` anomenada ``best_movie_rating`` ). 

In [27]:
# la vostra solució aquí
mean_ratings = data.pivot_table(values= 'rating', index=['title'], aggfunc='mean') # Busquem la mitjana de les puntuacions per cada pel·lícula
best_movie_rating = mean_ratings.sort_values(by='rating', ascending=False).iloc[0].name # Busquem el nom la pel·lícula amb la mitjana més alta



print(best_movie_rating)

Ulysses (Ulisse) (1954)


+ Mira si hi ha més pel·licules amb la mateixa puntuació de la més ben puntuada.

In [28]:
# la vostra solució aquí

better_rated = mean_ratings.sort_values(by='rating', ascending=False) # Ordenem les pel·lícules per rating
movies_with_highest_rating = better_rated.index[better_rated['rating'] == better_rated.loc[best_movie_rating]['rating']] # Busquem les pel·licules amb ratings iguals a la millor pel·lícula

movies_with_highest_rating

Index(['Ulysses (Ulisse) (1954)', 'Lured (1947)', 'Follow the Bitch (1998)',
       'Bittersweet Motel (2000)', 'Song of Freedom (1936)',
       'One Little Indian (1973)', 'Smashing Time (1967)',
       'Schlafes Bruder (Brother of Sleep) (1995)',
       'Gate of Heavenly Peace, The (1995)', 'Baby, The (1973)'],
      dtype='object', name='title')

+ Busca ara aquella pel·lícula, d'entre les que tenen 5 com a puntuació mitjana, que hagi rebut més valoracions i guarda-la a una variable anomenada ``best_movie_rating_maxviews``. Així tindrem la pel·licula més ben puntuada per més usuaris. 

In [29]:
# la vostra solució aquí

number_of_ratings = data.groupby('title').size() # Busquem el nombre de ratings per cada pel·lícula
# Busquem la pel·lícula amb més visualitzacions d'entre les de rating ==5
best_movie_rating_maxviews = pd.DataFrame(number_of_ratings.loc[movies_with_highest_rating].sort_values(ascending=False) ).iloc[0].name
best_movie_rating_maxviews


'Gate of Heavenly Peace, The (1995)'

### 3.2. EXERCICI B

+ Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada.


In [30]:
def top_movie(dataFrame,usr):
    # la vostra solució aquí
    return dataFrame[dataFrame['user_id'] == usr].sort_values(by='rating', ascending=False).iloc[0]['title']
    
    
print(top_movie(data,1))

One Flew Over the Cuckoo's Nest (1975)


### 3.3. EXERCICI C

+ Construeix una funció que donat el dataframe ``data`` et retorni un altre dataframe ``df_counts``amb el valor que cada usuari li ha donat a una peli. Això ho farem creant un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors siguin el rating donat.

In [31]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors
    la valoració que un usuari ha donat a una peli d'un `movie_id`
    
    :param df: DataFrame original 
    :return: DataFrame descrit adalt
    """
    
    # la vostra solució aquí
    return df.pivot_table(values= 'rating', index='user_id', columns='movie_id')
    
    
    

In [32]:
df_counts = build_counts_table(data)
df_counts

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


+ Fés una funció que donada la taula anterior i dos id's (usuari i peli), extregui el valor donat:

In [33]:
def get_count(df, user_id, movie_id):
    """
    Retorna la valoració que l'usuari 'user_id' ha donat de 'movie_id'
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param movie_id: ID de la peli
    :return: Enter amb la valoració de la peli
    """
    
    # la vostra solució aquí
    return df.loc[user_id][movie_id]
    
    
    

get_count(df_counts, 1, 1)

5.0

### 3.4. EXERCICI D

In [34]:
data.nunique()

user_id         6040
movie_id        3706
rating             5
timestamp     458455
gender             2
age                7
occupation        21
zip             3439
title           3706
genres           301
dtype: int64

In [35]:
unique_movies = pd.unique(data['movie_id'])
unique_movies.max()

3952

Si observem el nombre total d'usuaris únics i de pel.licules úniques, podem veure que els id's dels usuaris van de 1 a 6040. Normalment volem índexos que comencin al nombre 0, anant de 0 a 6039. 

+ Explora els índexos de les pel·licules. **Quin problema hi ha amb els indexos de les pel·licules??**

> **Resposta**:
> El problema que tenim amb els indexos de les pel·lícules és que en tenim 3706 de diferents però aquests van de 1 a 3706. Per tant hi ha més de 200 indexos en aquest rang que no estan assignats a cap pel·lícula.

+ Usant la funció `pd.Categorical(*).codes`, re-indexa els id's dels usuaris i de les pelis perquè vagin de 0 a 6039 i de 0 a 3705 respectivament:

In [36]:
# la vostra solució aquí

data['user_id'] = pd.Categorical(data['user_id']).codes #Reindexem els user_id

data['movie_id'] = pd.Categorical(data['movie_id']).codes #Reindexem els movie_id




+ Per comprovar que tot sigui correcte i guardar correctament la taula **df_counts**, torna a calcular i visualitza ``df_counts``:

In [37]:
# la vostra solució aquí
df_counts = build_counts_table(data)
df_counts

movie_id,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.5. EXERCICI E



+ Escriu una funció `distEuclid(x,y)`  que implementi la distància Euclidiana entre dos vectors usant funcions de pandas. 

+ Escriu la funció `SimEuclid (DataFrame, U1, U2)` que calculin la semblança entre dos usuaris segons aquesta estructura:

    + Calcular un vector per cada usuari, $U1$ i $U2$, amb les puntuacions dels ítems comuns que han puntuat el dos usuaris.
    + Si no hi ha puntuacions en comú, retornar 0. Sinó, retornar 
    
    $$\frac{1}{(1+distEuclid(U1, U2))}$$

+ Avalueu amb la funció ``%timeit`` quant triguen aquests càlculs per un parell d'usuaris.   

> *Nota: Alguns d'aquests exercicis tenen temps de càlcul de l'ordre de minuts sobre tota la base de dades. Per desenvolupar els algorismes és recomanable treballar amb una versió reduïda de la base de dades.* 

Per implementar aquestes funcions únicament es permet l'ús de les funcions:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**!

In [38]:
num_movies = data.nunique()['movie_id']

def distEuclid(x, y):
    """
    Retorna la distancia euclidiana de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distancia euclidiana
    """
    
    # la vostra solució aquí
    y = y.dropna() #Eliminem els valors NaN
    x = x.dropna()
    common_columns = x.index.intersection(y.index) #Busquem els índexs comuns
    if common_columns.empty:
        return np.nan
    x = x[common_columns] #Agafem només els índexs comuns
    y = y[common_columns]

    return np.linalg.norm(x-y)
    
    

def SimEuclid(DataFrame, User1, User2):
    """
    Retorna un score que representa la similitud entre user1 i user2 basada en la distancia euclidiana
    
    :param DataFrame: dataframe que conté totes les dades
    :param User1: id user1
    :param User2: id user2
    :return : Escalar (float) corresponent al score
    """
    # la vostra solució aquí
    #Fem servir df_counts en lloc de DataFrame perquè cada fila és un usuari i cada columna una pel·lícula
    x = df_counts.loc[User1]
    y = df_counts.loc[User2]
    dist = distEuclid(x,y)
    
    if pd.isna(dist):
        return 0
    else:
        return 1/(1+dist)
    

    

In [39]:
# Execute functions
print(SimEuclid(data, 2,314))

1.0


In [40]:
# Execute functions
print(SimEuclid(data, 0,1))

0.3333333333333333


In [41]:
# Execute functions
print(SimEuclid(data, 1, 5))

0.179128784747792


In [42]:
%timeit SimEuclid(data, 1, 5)

1.03 ms ± 23.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 3.6. EXERCICI F

En aquest exercici desenvoluparem un sistema de recomanació col·laboratiu **basat en usuaris**. 

La funció principal, ``getRecommendationsUser``, ha de tenir com a entrada una taula de puntuacions, un ``user_id``, el tipus de mesura de semblança (Euclidiana) que volem usar, el nombre `m` d'usuaris semblants que volem per fer la recomanació i el nombre ``n`` de recomanacions que volem. 

Exemple: ``getRecommendationsUser(data, 2, 50, 10, SimEuclid)``

Com a sortida ha de donar la llista de les $n$ millors pel·lícules que li podriem recomanar segons la seva semblança amb altres usuaris.

> *Nota 1: S'ha d'evitar comparar ``user_id`` a ell mateix.*

> *Nota 2: Recordeu que en Python podem passar funcions com a paràmetres d'una funció.*

#### EXERCICI F.1

+ Computa la *score* de similitud del usuari desitjat (userID) respecte tots els altres i retorna un diccionari dels $m$ usuaris més propers i el seu *score*, que seran els que usarem per fer la recomanació. Normalitzeu els *scores* de sortida.

In [43]:
def find_similar_users(DataFrame, userID, m, simfunction):
    """
    Retorna un diccionari de usuaris similars amb les scores corresponents.
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param similarity: mesura de similitud
    :return : dictionary
    """
    # la vostra solució aquí
    sim_users = pd.DataFrame(columns=['user_id', 'similarity']) #Creem un dataframe buit
    sim_users['user_id'] = DataFrame['user_id'].unique() #Afegim els user_id
    sim_users['similarity'] = sim_users['user_id'].apply(lambda x: simfunction(DataFrame, userID, x)) #Afegim les similituds
    sim_users = sim_users.sort_values(by='similarity', ascending=False).iloc[0:m] #Ordenem i agafem els m primers
    return dict(zip(sim_users['user_id'], sim_users['similarity'])) #Retornem un diccionari amb els user_id i les similituds
    

In [44]:
t = datetime.datetime.now()
sim_dict = find_similar_users(data, 2, 10, SimEuclid)
t = datetime.datetime.now()-t
print(str(t))

0:00:06.301444


In [45]:
sim_dict

{3069: 1.0,
 1239: 1.0,
 3583: 1.0,
 1438: 1.0,
 2790: 1.0,
 2182: 1.0,
 1481: 1.0,
 3679: 1.0,
 4184: 1.0,
 1618: 1.0}

+ Creieu que el temps de procés és assumible? Quan trigaria si ho fem per tots els usuaris?

> **RESPOSTA**:   El temps del procés no és assumible si s'ha d'executar regularment. Triga 3.2 segons per un usuari, això vol dir que pels 6040 usuaris que tenim al dataframe, trigaria més de 5 hores.

+ Per solucionar el problema anterior, construeix una matriu de mida $U \times U$ on cada posició $(i,j)$ indiqui la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3. Calcula quant triga la teva implementació. 

Heu de tenir en compte que:

* Si feu una funció que treballi amb els vectors de cada usuari i faci un doble ``for``, el procés de les dades pot trigar una bona estona.
* Si feu una funció que treballi específicament amb matrius (i no vectors) trigarà molt pocs segons. En aquest link podeu trobar indicacions de com fer-ho: https://jaykmody.com/blog/distance-matrices-with-numpy/



In [46]:
def compute_distance(fixed_arr, var_arr):
    """
    Donats dos vectors, calcula la distancia entre els subvectors formats 
    pels elements en comú. 
    """
    
    # la vostra solució aquí
    fixed_arr = fixed_arr.dropna() #Eliminem els valors NaN
    var_arr = var_arr.dropna()
    common_columns = fixed_arr.index.intersection(var_arr.index) #Busquem els índexs comuns
    if common_columns.empty:
        return np.nan
    fixed_arr = fixed_arr[common_columns] #Agafem només els índexs comuns
    var_arr = var_arr[common_columns]
    
    return np.linalg.norm(fixed_arr-var_arr)
    
    
    

In [47]:

def similarity_matrix(compute_distance, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param df_counts: df amb els valor que cada usuari li ha donat a una peli.
    :return : Matriu numpy de mida M x M amb les similituds.
    """
    
    # la vostra solució aquí
    df_counts_filled = df_counts.fillna(0)
    is_nan = np.isnan(df_counts.values) #Busquem els valors NaN
    is_nan = np.where(is_nan, 0, 1) #Els substituim per 0 (si és NaN) o 1 (si no és NaN)

    x2 = df_counts_filled.values**2 #Suma de les files al quadrat
    
    
    x2 = np.matmul(x2, is_nan.T) #Multiplicació de la matriu per la seva transposada

    #Aquí volem trobar la matriu de NaNs multiplicada per ella mateixa. Si hi ha un 0 vol dir que els usuaris no tenen cap pel·lícula en comú
    #Per alguna raó fer el següent funciona, però multiplicar is_nan per is_nan.T directament, fa que trigui excessivament
    identity = np.identity(is_nan.shape[1])
    
    nan_mat = np.matmul(identity, is_nan.T)
    nan2 = np.matmul(is_nan, nan_mat)

    xy = np.matmul(df_counts_filled.values, df_counts_filled.values.T) #Multiplicació de la matriu per la seva transposada
  
    dist = np.sqrt(x2 + x2.T - 2*xy ) #Fórmula de la similitud
    
    dist = np.where(nan2 == 0, np.nan, dist)
    similarity_matrix = pd.DataFrame(1/(1+dist))  #Similitud
    identity = np.identity(similarity_matrix.shape[0])
    similarity_matrix.fillna(0, inplace=True) #Substituim els NaN per 0
    similarity_matrix -= identity #Posem a 0 els elements de la diagonal
    return similarity_matrix
    
    

In [48]:
t = datetime.datetime.now()
sim = similarity_matrix(compute_distance, df_counts)
t = datetime.datetime.now()-t 
print("Temps amb doble for:",str(t)) #No fem servir doble for però no voliem canviar el que posa

Temps amb doble for: 0:00:12.273112


In [49]:
sim

,0,1,2,3,4,5,6,7,8,9,...,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039
0,0.000000,0.333333,0.231662,0.414214,0.139579,0.261204,0.500000,0.261204,0.261204,0.144591,...,0.250000,0.366025,1.000000,0.500000,0.240253,0.132320,0.231662,0.000000,0.205213,0.190744
1,0.333333,0.000000,0.205213,0.217129,0.100560,0.179129,0.161390,0.116961,0.179129,0.102302,...,0.231662,0.179129,0.205213,0.500000,0.104141,0.090094,0.152259,0.309017,0.309017,0.097337
2,0.231662,0.205213,0.000000,0.309017,0.175734,0.224009,0.333333,0.195194,0.224009,0.132320,...,0.231662,0.274292,0.224009,0.000000,0.179129,0.121786,0.200000,0.500000,0.217129,0.123899
3,0.414214,0.217129,0.309017,0.000000,0.231662,1.000000,0.240253,0.366025,0.414214,0.156613,...,0.274292,0.240253,0.190744,0.000000,0.175734,0.186605,0.289898,0.240253,0.333333,0.186605
4,0.139579,0.100560,0.175734,0.231662,0.000000,0.172538,0.116961,0.090909,0.101131,0.084959,...,0.169521,0.414214,0.139579,0.414214,0.112689,0.064125,0.114338,0.190744,1.000000,0.096830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.132320,0.090094,0.121786,0.186605,0.064125,0.123899,0.129732,0.076194,0.092610,0.047619,...,0.099449,0.111111,0.096331,0.261204,0.054243,0.000000,0.070468,0.138026,0.107450,0.060755
6036,0.231662,0.152259,0.200000,0.289898,0.114338,0.182744,0.163961,0.141188,0.138026,0.095358,...,0.142857,0.119782,0.224009,0.231662,0.115192,0.070468,0.000000,0.333333,0.138026,0.080350
6037,0.000000,0.309017,0.500000,0.240253,0.190744,0.289898,0.000000,1.000000,0.414214,0.274292,...,0.224009,0.205213,0.500000,0.000000,0.414214,0.138026,0.333333,0.000000,0.309017,0.138026
6038,0.205213,0.309017,0.217129,0.333333,1.000000,0.200000,1.000000,0.274292,0.414214,0.104778,...,0.144591,0.128496,0.414214,0.500000,0.179129,0.107450,0.138026,0.309017,0.000000,0.131006


+ Ara torna a re-fer la funció ``find_similar_users`` i mira quant triga. 

> Recorda que les scores han d'estar normalitzades!

In [50]:
def find_similar_users(DataFrame, sim_mx, userID, m):
    # la vostra solució aquí
    sim_users = sim_mx.iloc[userID] #Ordenem i agafem els m primers
    sim_users = sim_users.sort_values(ascending=False).iloc[0:m]

    return sim_users.to_dict() #Retornem un diccionari amb els user_id i les similituds

   
    
    
    

In [51]:
t = datetime.datetime.now()
sim_dict = find_similar_users(data, sim, 2, 10)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.005847


In [52]:
sim_dict

{1538: 1.0,
 5834: 1.0,
 5827: 1.0,
 152: 1.0,
 1831: 1.0,
 1051: 1.0,
 5816: 1.0,
 1839: 1.0,
 4648: 1.0,
 4650: 1.0}

#### EXERCICI F.2

+ Computa les recomanacions per un usuari concret a partir dels ratings dels seus $m$ usuaris més propers. Fes primer una funció que retorni la **weighted average list** dels $m$ usuaris més propers. Feu servir la funció anterior que usava la matriu de similituds per anar més ràpid!! (Nota: la **weighted average list** es calcularà agregant els $n$ items més puntuats de cadascun dels m users més semblants al usuari donat).

In [53]:

def weighted_average(DataFrame, user, sim_mx, m, df_counts):
    """
    df_counts: usuaris_semblants * pelis
    similarity: {user: similarity}
    
    Aquesta funcio calculara el score predit (Score que ha donat l'usuariB * similitud(A,B)) i despres sumara
    tots els valors per cada peli; retornara {peli_id: suma de score predit}
    """
    # la vostra solució aquí
    sim_users = find_similar_users(DataFrame, sim_mx, user, m) #Busquem els usuaris similars 
    
    scores = pd.DataFrame(columns=['movie_id', 'score']) #Creem un dataframe buit
    
    scores['movie_id'] = DataFrame['movie_id'].unique() #Afegim els movie_id
    
    scores['score'] = np.nanmean([df_counts.loc[user_id, scores['movie_id']] * similarity for user_id, similarity in sim_users.items()], axis=0) #Afegim els scores
       
    return dict(zip(scores['movie_id'], scores['score'])) #Retornem un diccionari amb els movie_id i els scores
        


In [54]:
def getRecommendationsUser(DataFrame, user, sim_mx, n, m):
    """
    Retorna un dataframe de pel·licules amb els scores.
    
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_function
    :param n: nombre de pelis a recomanar
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return : pandas de pelis amb els seus scores predits
    """
    # la vostra solució aquí
   
    scores = weighted_average(DataFrame, user, sim_mx, m, df_counts) #Busquem els scores
    
    scores = pd.DataFrame.from_dict(scores, orient='index', columns=['score']) #Creem un dataframe amb els scores
    
    scores.index.name = 'movie_id'   
    
    scores = scores.sort_values(by='score', ascending=False).iloc[:n] #Ordenem i agafem els n primers
    
    scores = scores.merge(movies,how='left', left_on='movie_id',right_on='movie_id') #Afegim els títols
    
    return scores
    

    

In [55]:
t = datetime.datetime.now()
user_prediction = getRecommendationsUser(data, 3, sim, 10, 50)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.965581


C:\Users\diego\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: Mean of empty slice
  app.launch_new_instance()


In [56]:
user_prediction

,movie_id,score,title,genres
0,1845,5.0,Zero Effect (1998),Comedy|Thriller
1,810,5.0,Kazaam (1996),Children's|Comedy|Fantasy
2,1175,5.0,Delicatessen (1991),Comedy|Sci-Fi
3,3203,5.0,Dead Calm (1989),Thriller
4,3274,5.0,Single White Female (1992),Action
5,1132,5.0,Manon of the Spring (Manon des sources) (1986),Drama
6,108,5.0,Catwalk (1995),Documentary
7,1288,5.0,This Is Spinal Tap (1984),Comedy|Drama|Musical
8,2059,5.0,"Parent Trap, The (1998)",Children's|Drama
9,1941,5.0,Hamlet (1948),Drama


### 3.7. EXERCICI G


A continuació usarem la metrica **Mean Absolute Error (MAE)** per evaluar el nostre sistema. Aquesta mètrica ens permetrà mesurar la diferencia entre dues llistes donat un usuari: 
+ La llista amb els ratings originals d'un usuari donat
+ La llista de les prediccions generades per aquest usuari

#### EXERCICI G.1

+ Treu el 10% dels usuaris i reserva aquests en una variable anomenada ``test_set`` i la resta en una variable anomenada ``train_set``.

In [57]:
# la vostra solució aquí
test_set = data.sample(frac=0.1, random_state=0) #Agafem el 10% de les dades
train_set = data.drop(test_set.index) #Eliminem el 10% de les dades


assert len(test_set) + len(train_set) == len(data)

+ Què passarà si calculo la matriu de similitud amb ``train_set`` i després intento predir pels usuaris de ``test_set``??

> **Resposta**: Si hem fet bé el càlcul de similitud, probablement els valors seran similars a les valoracions dels usuaris de test_set


#### EXERCICI G.2

+ Selecciona aproximadament el 80% de les interaccions de cada usuari de test i junta-les al ``train_set``. Podem ara podem evaluar el sistema?


> Com la pràctica és molt llarga us donem el codi per un usuari donat i vosaltres només heu de crear la funció que, per cada usuari, afagi el 80% de les intraccions i les unifiqui al dataframe de train.

In [58]:
test_set.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
324271,1995,2213,4,974743570,M,35,0,85621,Romancing the Stone (1984),Action|Adventure|Comedy|Romance
818637,920,1890,4,975203825,M,25,17,92084,And the Band Played On (1993),Drama
148677,322,3566,4,976408046,M,45,12,53716,What About Bob? (1991),Comedy
778790,345,1291,4,976335541,F,25,0,55110,My Fellow Americans (1996),Comedy
525489,4003,2958,2,965560544,M,50,0,21050,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi


In [59]:
# Agafem el 20% de les pelis que ha consumit cada usuari de test 
groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2
groupby_count

user_id
0        1.0
1        3.2
2        0.2
3        0.2
4        5.0
        ... 
6035    19.0
6036     5.0
6037     0.4
6038     2.8
6039     6.8
Name: movie_id, Length: 5971, dtype: float64

Seleccionem la posició 1 i aquest use_id serà el que usarem pel codi d'exemple (que després haureu de replicar).

In [60]:
groupby_count.reset_index().iloc[1]

user_id     1.0
movie_id    3.2
Name: 1, dtype: float64

In [61]:
n_test_samples = int(groupby_count.reset_index().iloc[1]['movie_id'])
u = groupby_count.reset_index().iloc[1]['user_id']

In [62]:
test_set_user = test_set[test_set['user_id'] == u]
frame_test = test_set_user.sample(n_test_samples)
print("TOTAL SAMPLES OF THE USER: " + str(len(test_set_user)))
print("TOTAL SAMPLES OF THE USER IN TEST SET: " + str(len(frame_test)))

TOTAL SAMPLES OF THE USER: 16
TOTAL SAMPLES OF THE USER IN TEST SET: 3


In [63]:
len(test_set_user.index)

16

In [64]:
frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]
print("TOTAL SAMPLES OF THE USER IN TRAIN SET: " + str(len(frame_train)))

TOTAL SAMPLES OF THE USER IN TRAIN SET: 13


In [65]:
assert len(frame_train) + len(frame_test) == len(test_set_user)

In [66]:
def add_testdata(traindf, test_set):
    """
    Retorna els N usuaris més similars basat en la correlació de Pearson (no)
    
    :param traindf: dataframe que conté les dades de train
    :param test_set: dataframe que conté les dades de test

    :return : 
        - :param 1st: dataframe que conté les dades de train juntament amb el 80% de test seleccionat
        - :param 2nd: dataframe que conté les dades de test que queden (20% restant)
    """
    groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2

    # la vostra solució aquí
    train_with_test = traindf.copy()
    remaining_test = pd.DataFrame()
    unique_user_ids = test_set['user_id'].unique()
    # Iterem sobre els usuaris
    for user_id in tqdm(unique_user_ids, desc='Processing Users'):

        
        
        test_set_user = test_set[test_set['user_id'] == user_id] #Agafem les dades de l'usuari actual

        n_test_samples = int(groupby_count.loc[user_id])
        
        frame_test = test_set_user.sample(n_test_samples)
        frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]

        # Afegim els dataframes
        train_with_test = pd.concat([train_with_test, frame_train])
        remaining_test = pd.concat([remaining_test, frame_test])


    return train_with_test, remaining_test
   
    

In [88]:
train, test = add_testdata(train_set, test_set)

Processing Users:   0%|          | 0/5971 [00:00<?, ?it/s]

In [68]:
train

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,0,1104,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,1104,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,11,1104,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,14,1104,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,16,1104,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
952653,1843,98,5,974696201,F,18,4,53711,Bottle Rocket (1996),Comedy
853451,5390,2941,1,960324450,M,25,7,10016,Stuart Little (1999),Children's|Comedy
77567,4526,3341,5,964801421,M,18,0,38111,Gladiator (2000),Action|Drama
76210,352,3341,4,976329601,F,35,7,92625,Gladiator (2000),Action|Drama


In [69]:
test

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
450408,1995,307,4,974742803,M,35,0,85621,Stargate (1994),Action|Adventure|Sci-Fi
460939,1995,1980,4,974742676,M,35,0,85621,"NeverEnding Story, The (1984)",Adventure|Children's|Fantasy
929696,1995,352,4,974751322,M,35,0,85621,"Jungle Book, The (1994)",Adventure|Children's|Romance
83550,920,1478,4,975203825,M,25,17,92084,"Hunt for Red October, The (1990)",Action|Thriller
102456,920,575,4,975201546,M,25,17,92084,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
...,...,...,...,...,...,...,...,...,...,...
913051,1572,3327,3,974737751,M,1,0,77479,"Flintstones in Viva Rock Vegas, The (2000)",Children's|Comedy
445404,1404,1184,4,974766777,M,56,13,33445,Cool Hand Luke (1967),Comedy|Drama
238586,417,46,5,976286068,F,25,3,54016,Seven (Se7en) (1995),Crime|Thriller
643884,3866,595,3,965848437,M,35,3,45215,Heavy Metal (1981),Action|Adventure|Animation|Horror|Sci-Fi


In [70]:
train.shape

(982660, 10)

In [71]:
test.shape

(17549, 10)

In [72]:
data.shape

(1000209, 10)

In [73]:
assert train.shape[0] + test.shape[0] == data.shape[0]

#### EXERCICI G.3

+ Fes una funció que serveixi per evaluar el nostre sistema usant la mètrica MAE. 

In [74]:
import warnings
    
def evaluateRecommendations(train, test, m, n, sim):
    """
    Retorna l'error generat pel model
    
    :param train: dataframe que conté les dades de train
    :param test: dataframe que conté les dades de test
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param n: nombre de pelis a retornar
    :param sim: matriu de similitud
    :return : Escalar (float) corresponent al MAE
    """
    
    # la vostra solució 
    mae = 0
    count = 0
    unique_user_ids = test['user_id'].unique()
    for user_id in tqdm(unique_user_ids, desc='Testing Users'): #Iterem sobre els usuaris de test
        
        test_user = test[test['user_id'] == user_id] #Agafem les dades de l'usuari actual
        #print(test_user)
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        #Busquem els scores a partir de les dades de train
        predicted_ratings = getRecommendationsUser(train, user_id, sim, n, m)[['movie_id','score']] 
        #print('predicted_ratings: ', predicted_ratings)
        actual_ratings = test_user[['movie_id','rating']]#Agafem les valoracions reals
        
        #Agafem les prediccions dels valors reals dels que tenim dades (habitualment tenim més recomanacions que valoracions reals)
        predicted_of_actual = predicted_ratings[predicted_ratings['movie_id'].isin(actual_ratings['movie_id'].values)][['movie_id', 'score']]
        #print('predicted of actual : \n' , predicted_of_actual )
        #print('actual_ratings ' , actual_ratings)
        predicted_of_actual = predicted_of_actual.set_index('movie_id') #Reindexem per fer la diferència correctament
        actual_ratings = actual_ratings.set_index('movie_id')
        abs_diff = np.abs(predicted_of_actual['score'] - actual_ratings['rating']) #Trobem la difèrencia absoluta

        #print('abs diff' , abs_diff)
        user_mae = np.nanmean(abs_diff) #Calculem el MAE per l'usuari actual
        if not np.isnan(user_mae):
            count += 1
        mae += np.nansum(user_mae) #Afegim el MAE a la suma total
        
        
    #Calculem el MAE mitjà
    mae /= count
    return mae


In [89]:
t = datetime.datetime.now()
mae = evaluateRecommendations(train, test, 50, 10, sim)
t = datetime.datetime.now()-t
print(str(t))

Testing Users:   0%|          | 0/4439 [00:00<?, ?it/s]

0:04:11.253634


In [90]:
mae

0.27351163177207105

### 3.8. EXERCICI H (exercici opcional, no obligatori)


+ **Que surt més a compte, fer un recomanador unic pels dos sexes o un per cada sexe?** Justifica la resposta per escrit i amb el codi necessari.

Comparem el resultat MAE obtingut analitzant les dades totals amb les que obtenim diferenciant per sexe:

In [81]:
test_set_m = data[data['gender']=='M'].sample(frac = 0.1, random_state = 0)
train_set_m = data[data['gender']=='M'].drop(test_set_m.index)
assert len(test_set_m) + len(train_set_m) == len(data[data['gender']=='M'])

train_m, test_m = add_testdata(train_set_m, test_set_m)

mae_m = evaluateRecommendations(train_m, test_m, 50, 10, sim)

Processing Users:   0%|          | 0/4267 [00:00<?, ?it/s]

Testing Users:   0%|          | 0/3244 [00:00<?, ?it/s]

In [82]:
mae_m

0.17355963739975996

In [91]:
test_set_f = data[data['gender']=='F'].sample(frac = 0.1, random_state = 0)
train_set_f = data[data['gender']=='F'].drop(test_set_f.index)
assert len(test_set_f) + len(train_set_f) == len(data[data['gender']=='F'])

train_f, test_f = add_testdata(train_set_f, test_set_f)

mae_f = evaluateRecommendations(train_f, test_f, 50, 10, sim)

Processing Users:   0%|          | 0/1678 [00:00<?, ?it/s]

Testing Users:   0%|          | 0/1192 [00:00<?, ?it/s]

In [92]:
mae_f

0.1039970113194778

Com es pot veure, els errors absoluts mitjans dins dels sexes són menors que els del conjunt, indicant una similitut de gustos. Per tant, creiem que emprar dos recomanadors, un per sexe, pot disminuïr l'error comés al fer la recomanació, és a dir, farà recomanacions més especialitzades i adequades a l'usuari.